# Network graph for a Bitcoin Lightning Network routing node

To create a network graph based on relative transactional volume and related fees between channels.

Date: 11/02/2023

Fulmine Labs LLC

Under MIT open source license

The problem: It can be hard for Lightning Network node operators to determine which of their channels are performing well (and which may not be) from the raw transactional data provided by interfaces such as Ride the Lightning (RTL).

This Python code aggregates transactions and allows at-a-glance vizualization of where traffic was heavy or light, which channels cluster together and how fees were set relative to other nodes.

The input to this is a Ride the Lightning transaction history CSV file, exported from the RTL routing interface
A sample file, Forwarding-history-sample.csv is provided. The sample data has been anonymized for privacy, but actual RTL data will display actual node names.
Modify the call to load_data(), as needed.

1. Node Creation: Each node represents a channel that has participated in some routing
2. Edge Creation: Each edge between two nodes represents the transactional volume between those two channels, where the thickness or color of the edge represents the normalized transactional volume
3. Node Size: The size of the node indicates the sum of the transactions for that channel
4. Node Color: The node color indicates the average fee for that node:
 *   Red no outbound transactions
 *   Orange lowest fees
 *   Yellow medium fees 
 *   Green highest fees

To achieve this, we used the networkx library, a tool for creating and analyzing networks and the ipycytoscape library to visualize and interact with the networkx graph.
cytoscape allows interactive zoom and the dragging of nodes for improved visualization

This code was run in Jupyter Notebook and Jupyter Lab from Anaconda on Windows. It was written collaboratively with GPT-4V. Thank you Assistant!

It was tested with over 10,000 transactions. The graph got a little 'dense' (GPT-4V's description), but is definitely still helpful, particularly by taking advantage of cytoscape's interactive capabilties. 

To avoid 'datarate is exceeded' errors in Jupyter for large numbers of transactions, it is recommended to start Jupyter from Anaconda Powershell:
_jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10_
or
_jupyter lab --NotebookApp.iopub_data_rate_limit=1.0e10_

If successful the output should look something like this:



![alt text](sample_output.png "Title")


In [1]:
import networkx as nx
import pandas as pd
import ipycytoscape
import csv
import sys

In [2]:
# Load the RTL data from the CSV file
def load_data(filepath):
    try:
        # Attempt to read the CSV file
        return pd.read_csv(filepath, encoding='utf-8')
    except FileNotFoundError:
        # Handle the case where the file doesn't exist
        print(f"Error: The file {filepath} was not found.")
        return None
    except PermissionError:
        # Handle the case where the file exists but you don't have the permission to open it
        print(f"Error: Permission denied when trying to read the file {filepath}.")
        return None
    except pd.errors.EmptyDataError:
        # Handle the case where the file is empty
        print(f"Error: The file {filepath} is empty.")
        return None
    except pd.errors.ParserError:
        # Handle the case where the file has parsing issues
        print(f"Error: The file {filepath} does not appear to be a valid CSV or is improperly formatted.")
        return None
    except Exception as e:
        # Handle any other exception and print out a message
        print(f"An unexpected error occurred: {e}")
        return None

In [3]:
# Add nodes and edges and update edge weights 
def add_nodes_and_edges(G, df):
    """Add nodes and edges to the graph G based on the dataframe df."""
    for index, row in df.iterrows():
        
        # Add nodes for channels if they don't already exist
        G.add_node(row['alias_in'])
        G.add_node(row['alias_out'])

        # Check if there's already an edge (i.e., a previous transaction between these channels)
        if G.has_edge(row['alias_in'], row['alias_out']):
            edge_data = G[row['alias_in']][row['alias_out']]
            edge_data['weight'] += row['amt_out_msat']
            edge_data['fees'].append(row['fee_msat'])
        else:
            # Convert both node names to strings (deal with case that they can be interpreted as integers)
            u = str(row['alias_in'])
            v = str(row['alias_out'])
            G.add_edge(u, v, weight=row['amt_out_msat'], fees=[row['fee_msat']])
    
    # Now, average out the fees and compute total transactions once, for efficiency
    total_transactions = sum(nx.get_edge_attributes(G, 'weight').values())
    for u, v, d in G.edges(data=True):
        d['avg_fee'] = sum(d['fees']) / len(d['fees'])
        d['normalized_fee'] = d['avg_fee'] / total_transactions * 10000        
        
    return (total_transactions)


In [4]:
# Assign classes to nodes based on normalized transactions and fees
def assign_classes_to_nodes (cyto_graph, df):

    for node in cyto_graph.graph.nodes:
        node_alias = node.data['id']

        classes = []
        
        # # Calculate total fees and total transactions for each node
        total_fees = df.groupby('alias_out')['fee_msat'].sum()
        total_transactions_gr = df.groupby('alias_out')['amt_out_msat'].sum()

        max_transaction = total_transactions_gr.max()

        # Normalize the transactions
        normalized_transactions_node = (total_transactions_gr / max_transaction).to_dict()
        normalized_fees = (total_fees / total_transactions_gr * 10000).to_dict()

        # Assign fee classes
        if node_alias in normalized_fees:

            fee = normalized_fees[node_alias]
            if fee <= 0.1:
                classes.append('low_fee')
            elif fee <= 0.2:
                classes.append('medium_fee')
            else:
                classes.append('high_fee')
        else:
            classes.append('no_fee')

        # Assign transaction classes
        if node_alias in normalized_transactions_node:
            trans = normalized_transactions_node[node_alias]
            if trans <= 0.1:
                classes.append('tiny_transaction')
            elif trans <= 0.5:
                classes.append('small_transaction')
            elif trans <= 0.9:
                classes.append('medium_transaction')
            else:
                classes.append('large_transaction')

        # Combine classes with space
        node.classes = ' '.join(classes)

In [5]:
# Assign classes to edges based on normalized transaction volumes
def assign_classes_to_edges (cyto_graph, G, total_transactions):

    for edge in cyto_graph.graph.edges:

        u, v = edge.data['source'], edge.data['target']   
        d = G[u][v]

        normalized_transaction = d['weight']/total_transactions * 10000

        if (normalized_transaction < 1):
            edge.classes = 'tiny_volume'
        elif normalized_transaction < 10:
            edge.classes = 'small_volume'
        elif normalized_transaction < 100:
            edge.classes = 'medium_volume'
        else:
            edge.classes = 'large_volume'
        

In [6]:
# Set styles based on classes
def set_styles (cyto_graph):
    
    cyto_graph.set_style([
        {
            'selector': 'node.no_fee',
            'style': {
                'content': 'data(id)',
                'background-color': 'red'
            }
        },
        {
            'selector': 'node.low_fee',
            'style': {
                'content': 'data(id)',
                'background-color': 'orange'
            }
        },
        {
            'selector': 'node.medium_fee',
            'style': {
                'content': 'data(id)',
                'background-color': 'yellow'
            }
        },
        {
            'selector': 'node.high_fee',
            'style': {
                'content': 'data(id)',
                'background-color': 'green'
            }
        },
            {
            'selector': 'node.tiny_transaction',
            'style': {
                'width': '10px',
                'height': '10px'
            }
        },
        {
            'selector': 'node.small_transaction',
            'style': {
                'width': '20px',
                'height': '20px'
            }
        },
        {
            'selector': 'node.medium_transaction',
            'style': {
                'width': '30px',
                'height': '30px'
            }
        },
        {
            'selector': 'node.large_transaction',
            'style': {
                'width': '50px',
                'height': '50px'
            }
        },
        {
            'selector': 'edge.tiny_volume',
            'style': {
                'width': '0px',
                'line-color': 'grey'
            }
        },
        {
            'selector': 'edge.small_volume',
            'style': {
                'width': '1px',
                'line-color': 'grey'
            }
        },
        {
            'selector': 'edge.medium_volume',
            'style': {
                'width': '2px',
                'line-color': 'grey'
            }
        },
        {
            'selector': 'edge.large_volume',
            'style': {
                'width': '3px',
                'line-color': 'black'
            }
        }
    ])


In [7]:
# Create a new graph
G = nx.Graph()

#Note, the provided 5 days of sample data has been anonymized for privacy. Using actual RTL node data will show the node ids
df = load_data ("Forwarding-history-sample.csv")
#df = load_data ("Forwarding-history.csv")

# Check if the data was loaded successfully
if df is None:
    # If df is None, print an error message and exit
    print("Transaction data could not be loaded. Please check the file path and format.")
    sys.exit(1)

total_transactions = add_nodes_and_edges(G, df)

cyto_graph = ipycytoscape.CytoscapeWidget()
cyto_graph.graph.add_graph_from_networkx(G)

cyto_graph.set_layout(name='circle')

assign_classes_to_nodes (cyto_graph, df)
assign_classes_to_edges (cyto_graph, G, total_transactions)

set_styles (cyto_graph)

# Note: Start Jupyter with "jupyter notebook (or jupyter lab) --NotebookApp.iopub_data_rate_limit=1.0e10", in Powershell if datarate is exceeded
# This can happen with large numbers of transactions
cyto_graph

CytoscapeWidget(cytoscape_layout={'name': 'circle'}, cytoscape_style=[{'selector': 'node.no_fee', 'style': {'c…